# DGPC Geolocation

In [15]:
# classic import
import pandas as pd
import numpy as np
from shapely.geometry import Point, LineString, Polygon
from shapely import wkt
import geopandas as gp
import matplotlib.pyplot as plt
import warnings
import sys
import matplotlib as mpl
warnings.simplefilter('ignore')

In [16]:

# import local functions
path_to_functions = './plotting.py'
sys.path.append(path_to_functions)
from plotting import default_plot, get_colors

In [17]:
# matplotlib formatting
small = 10
medium = 12
large = 14

plt.rc('font', size=small)         # controls default text sizes
plt.rc('axes', titlesize=small)     # fontsize of the axes title
plt.rc('axes', labelsize=small)     # fontsize of the x and y labels
plt.rc('xtick', labelsize=small)   # fontsize of the tick labels
plt.rc('ytick', labelsize=small)   # fontsize of the tick labels
plt.rc('legend', fontsize=small)   # legend fontsize
plt.rc('figure', titlesize=large)   # fontsize of the figure title
mpl.rcParams['font.family'] = "sans-serif"
mpl.rcParams['font.sans-serif'] = "Arial"
mpl.rcParams['axes.linewidth'] = 0.5
mpl.rcParams['xtick.major.width'] = 0.5
mpl.rcParams['ytick.major.width'] = 0.5   

In [18]:
# define functions

def horizontal_bar(data, rotation=0, name='figures/default_categorical_bar', save=False):
    
    # makes a categorical bar plot
    # input data has to be a dataframe with 3 columns
    # column 0 contains the labels that go on the x-axis (already pre-formatted)
    # column 1 contains the values
    # column 2 contains the colors
    # the title of the value-columns contains the label for the Y-axis
    # the name variable should contain path and filename to save to file
    
    data = data[::-1]
    
    size_dict = {4:4,
                 17:8}
    
    labels = list(data[data.columns[0]])
    labels = [str(len(labels) - i) + '. ' + labels[i] for i in range(len(labels))]
    values = list(data[data.columns[1]])

    fig, ax = plt.subplots(figsize=(6, 6))
    ax = default_plot(ax, ['left'])
    x = range(len(values))
    
    ax.yaxis.set_tick_params(length=0)
    ax.spines['left'].set_visible(False)
    
    plt.ylim(-0.45, len(values) - 0.45)
    #plt.xlim(0, 1)

    xlabel = data.columns[1].replace('_',' ').title()
    ylabel = data.columns[0].replace('_',' ').title()

    plt.xlabel(xlabel)
    plt.yticks(x, labels)

    plt.vlines([10000, 20000, 30000], 0, 23, lw=.5, ls='dashed', alpha=.5, zorder=0)
    plt.barh(x, values, height=.9, color=data.colors, edgecolor='k', lw=.5)
    
    if save:
        plt.tight_layout()
        plt.savefig(name + '.png', dpi=100)
    
    plt.show()


In [27]:
# open data RUTE
concelhos = gp.read_file('dados/concelhos.shp', encoding='utf-8')
pontos_interesse = pd.read_csv('dados/wiki-patrimonio-cult-portugal.csv')

populacao = pd.read_excel('dados/PORDATA-INE-populacao-concelho-2018.xlsx')

#population = population[(population['CODIGO'] == "'1106'") & (population['NOME'] != 'Lisboa')].reset_index()


In [20]:
#pontos_interesse["geocoords"] = pontos_interesse["coords"].apply(lambda x: x.replace(' ',', '))

In [34]:
pontos_interesse.isnull().sum()

item            0
itemLabel       0
coords          0
image        2581
dgpcid          0
dtype: int64

In [41]:
pontos_interesse["coords"] = pontos_interesse["coords"].apply(wkt.loads)
#geo_df = gpd.GeoDataFrame(pontos_interesse, geometry='coords')

ParseException: Unknown type: 'T912295384'


WKTReadingError: Could not create geometry because of errors while reading input.

In [12]:
# add population values to freguesias dataset
pop_map = pd.Series(population.POP.values, index=population.NOME).to_dict()
freguesias['POP'] = freguesias['NOME'].map(pop_map)

NameError: name 'population' is not defined

In [13]:
# open data
freguesias = gp.read_file('data/freguesias.shp')
padarias = pd.read_csv('data/padarias.csv', index_col=0)
population = pd.read_csv('data/populacao-freguesias.csv', sep=';', engine='python', header=None, usecols=[1, 3, 4], names=['CODIGO','NOME', 'POP'])
population = population[(population['CODIGO'] == "'1106'") & (population['NOME'] != 'Lisboa')].reset_index()

# there were two duplicated restaurants in the same location
padarias.drop_duplicates(subset='restaurant.name', inplace=True)



DriverError: data/freguesias.shp: No such file or directory

In [ ]:
# change projection to 2D in meters
freguesias = freguesias.to_crs('EPSG:3763')

In [ ]:
# transform Zomato data into GeoDataFrame
padarias = gp.GeoDataFrame(padarias, geometry=gp.points_from_xy(padarias['restaurant.location.longitude'], padarias['restaurant.location.latitude']))

# transform from naive coordinates to the actual CRS system they are in.
padarias.crs = {'init' :'EPSG:4326'}

# re-project to the same as the Lisbon map
padarias = padarias.to_crs('EPSG:3763')

In [ ]:
# remove points outside the city
padarias = padarias[padarias.geometry.within(freguesias.unary_union)]

In [ ]:
# plot base map and restaurant locations
base = freguesias.plot(figsize=(6, 6), facecolor='#34495e', edgecolor='black', linewidth= 1.5, alpha=.1)
padarias.plot(ax=base, marker='o', color='#e74c3c', markersize=40, edgecolor='k', linewidth=.5)
base.axis('off');
base.margins(0)
base.tick_params(left=False, labelleft=False, bottom=False, labelbottom=False)
#plt.savefig('figures/basemap_locations.png', dpi=100, bbox_inches='tight', pad_inches=0)

In [ ]:
# compute the 500m radius
circles = padarias.buffer(500)

In [ ]:
# compute intersection between basemap and radiuses
intersection = circles.unary_union.intersection(freguesias.unary_union)

In [ ]:
# plot new map
base = freguesias.plot(figsize=(6, 6), facecolor='#34495e', edgecolor='black', linewidth= 1.5, alpha=.1)
gp.GeoSeries(intersection).plot(ax=base, facecolor='none', edgecolor='k', lw=1, ls='dashed')
gp.GeoSeries(intersection).plot(ax=base, facecolor='#e74c3c', alpha=.4)
base.axis('off');
base.margins(0)
base.tick_params(left=False, labelleft=False, bottom=False, labelbottom=False)
plt.savefig('figures/area_difference_thumb.png', dpi=100, bbox_inches='tight', pad_inches=0, transparent=True)

In [ ]:
# compute percentage of area covered
area_covered = intersection.area/freguesias.unary_union.area
print(area_covered)

In [ ]:
# calculate insersection for each freguesia
intersections_all = freguesias['geometry'].intersection(circles.unary_union)

In [ ]:
# compute area coverr for each freguesia
freguesias['area_covered'] = intersections_all.area/freguesias.area

In [ ]:
# calculate population reached
freguesias['population_reached'] = freguesias['POP'] * freguesias['area_covered']
freguesias.sort_values(by='population_reached', ascending=False, inplace=True, ignore_index=True)

# grab colors
freguesias['colors'] = get_colors('viridis_r', freguesias.shape[0], plot=False)

In [ ]:
# make categorical and horizontal barplot
horizontal_bar(freguesias[['NOME', 'population_reached', 'colors']], save=True)

In [ ]:
# get representative coordinate for each freguesia
freguesias['coords'] = freguesias['geometry'].apply(lambda x: x.representative_point().coords[:])
freguesias['coords'] = [coords[0] for coords in freguesias['coords']]

# plot annotated map
base = freguesias.plot(figsize=(6, 6), facecolor=freguesias['colors'], edgecolor='black', linewidth= .5, alpha=1)

# make number annotations
for idx, row in freguesias.iterrows():
    plt.annotate(s=str(idx+1), 
                 xy=row['coords'], 
                 horizontalalignment='center', 
                 bbox=dict(facecolor='white', edgecolor='black', lw=.3, boxstyle='round,pad=.3'))
    
base.axis('off');
base.margins(0)
base.tick_params(left=False, labelleft=False, bottom=False, labelbottom=False)
plt.savefig('figures/freguesias_choropleth.png', dpi=100, bbox_inches='tight', pad_inches=0, transparent=True)

In [ ]:
# compute population proportion
freguesias['population_reached'].sum()/freguesias['POP'].sum()

In [ ]:
freguesias['density'] = freguesias['POP']/freguesias['AREA_M2']

In [ ]:
freguesias[['NOME','density','area_covered']].sort_values(by='are')